In [ ]:
import godot

import numpy as np
from godot.core import tempo, util, constants
from godot.core import autodif as ad
from godot.core.autodif import bridge as br
from godot.model import common
from godot import cosmos
from tqdm import tqdm
import time


# optionally avoid verbose logging messages
import godot.core.util as util
util.suppressLogger()


# create the universe
uni_config = godot.cosmos.util.load_yaml('lunar_orbits.yml')
uni = godot.cosmos.Universe(uni_config)

# specify a time grid
ep1 = tempo.Epoch('2026-04-01T00:00:00 TDB')
ep2 = tempo.Epoch('2027-04-01T00:00:00 TDB')
ran = tempo.EpochRange(ep1, ep2)
eps = 1e-6
tol = 1e-6
event_grid = ran.contract(eps).createGrid(1.0)

from typing import Union
class CalculateBlock(common.ScalarTimeEvaluable):
    def __init__(self, uni, centre, radius, observer, target, reference):
        # We have to call C++ trampoline class constructor explicitly
        common.ScalarTimeEvaluable.__init__(self)
        self.uni = uni
        self.centre = centre
        self.radius = radius
        self.observer = observer
        self.target = target
        self.reference = reference

    def eval(self, epoch: Union[tempo.Epoch, tempo.XEpoch]) -> bool:
        SC = self.uni.frames.vector3(self.centre, self.target, self.reference, epoch)
        GS = uni.frames.vector3(self.centre, self.observer, self.reference, epoch)
        
        GSSC = GS-SC

        direction = (GSSC) /  ad.norm(GSSC)  # Normalize direction
        b = 2 * ad.dot(GS, direction)
        c = ad.dot(GS, GS) - ad.pow(self.radius,2)
        discriminant = ad.pow(b,2) - 4 * c
        if discriminant < 0:
            return True  # No intersection
        t_1 = (-b - ad.sqrt(discriminant))/2
        t_2 = (-b + ad.sqrt(discriminant))/2

        t = t_1 if ad.abs(t_1) < ad.abs(t_2) else t_2     
        intersection = GS + t * direction

        int_dist = ad.sqrt(sum(ad.pow(GS - intersection,2)))
        sc_dist = ad.sqrt(sum(ad.pow(GSSC,2)))
        #print(f"Intersection distance: {int_dist}, spacecraft distance: {sc_dist}, closer?: {sc_dist < int_dist}")
        closer = sc_dist < int_dist
        if closer:
            return True
        else:
            return False
    
NNE = []
CBE = []
MGE = []
# Construct a constant radial acceleraiton
NN_elevation = CalculateBlock(uni, 'Moon', 1737.4, 'NN11', 'SC', 'ICRF')
CB_elevation = CalculateBlock(uni, 'Moon', 1737.4, 'CB11', 'SC', 'ICRF')
#MG_elevation = CalculateBlock(uni, 'Moon', 1737.4, 'MG11', 'SC', 'ICRF')
for t in tqdm(event_grid):
    NNE.append(NN_elevation.eval(t))
    CBE.append(CB_elevation.eval(t))
    #MGE.append(MG_elevation.eval(t))

  1%|          | 197859/31536001 [00:16<43:41, 11954.70it/s]


KeyboardInterrupt: 